## Install

Fandango is installed as a python module and can be accessed through the command line. The easiest way to install is to run `pip install fandango-fuzzer`. After this, Fandango can be accessed through the terminal using `fandango` (or `python -m fandango`) in the same environment as where it was installed.

On Windows, if there are no pre-built wheels, Microsoft Visual Studio Build Tools and the Windows 10 SDK are required to build Fandango.

## Name Database (Demo)

You can create a database of name/age pairs by way of a grammar, stored in a .fan file as a set of rules dictating how the Fandango input should be generated. Note that <start> is the very first variable that the grammar should refer to.

Then, this command will generate 10 name, age pairs based on the grammar specified in persons.fan (an example .fan file).

`fandango fuzz -f persons.fan -n 10`

1. `-o FILENAME` redirects the output of the above command into a file FILENAME.
2. `-s DELIMITER` replaces the delimiter between outputs with DELIMITER (default is a newline).
3. `-d DIRECTORYNAME` stores each output in a directory named DIRECTORYNAME.

1. `fandango fuzz -f persons.fan -n 10 -o persons.txt`
2. `fandango fuzz -f persons.fan -n 10 -o persons.txt -s ':'`
3. `fandango fuzz -f persons.fan -n 10 -d persons`

The number of outputs can be limited via the numerical argument -n or -N, where -n refers to the number of solutions/outputs, and -N refers to the number of generated sets.

1. fandango fuzz -f persons.fan -n 10 # to limit the number of solutions
2. fandango fuzz -f persons.fan -N 10 # to limit the number of generations


## Specifying Constraints

Constraints can be written as boolean expressions, where "< VARNAME >" is used to refer to variable VARNAME. Note that all constraints should be preceded by the "where" keyword in grammars. 
where int(< age >) < 50 (age must be less than 50)

In addition to grammars, constraints can also be specified on the command line with -c to denote a constraint.

fandango fuzz -f persons.fan -n 10 -c '25 <= int(< age >) and int(< age >) <= 45'

How Fandango Solves Constraints
1. It first uses the grammar to generate a population of inputs.
2. It then checks which individual inputs are closest in fulfilling the given constraints. For instance, for a constraint int(<X>) == 100, an input where <X> has a value of 90 is closer to fulfillment than one with value of, say 20. Selecting the best inputs is also known as “survival of the fittest”.
3. The best inputs are selected, the others are discarded.
4. Fandango then generates new offspring by mutating the remaining inputs, recomputing parts according to grammar rules. It can also exchange parts with those from other inputs; this is called crossover.
5. Fandango then repeats Steps 2-4 until all inputs satisfy the constraints.

Soft constraints may also be used, which do not fix values but specify arbitrary minimums and/or maximums instead.
1. Ex. fandango fuzz -f persons.fan --maximize 'int(<age>)' (Maximizes age field)
2. Ex. fandango fuzz -f persons.fan --minimize 'int(<age>)' (Minimizes age field)



## Regular Expressions/Complex Inputs

Fandango allows users to instantiate strings from regexp templates, and use regexp to parse and match inputs.

By default, there is an imposed repetition limit of 20.

Example of RegExp grammar:
1. <start> ::= r"(abc)+"

fandango fuzz -f infinity.fan -n 10

Specify {m, n} to limit the number of repetitions to m to n
1. <start> ::= r"(abc){1,5}"

fandango fuzz -f finity.fan -n 5

Complex inputs can also be produced in Fandango, like recursive ones:
Ex. fandango fuzz -f additions.fan -n 10

Need to specify non-recursive terms to prevent infinite recursion

Recursive rules can substitute for "+"
1. Ex. <number> ::= <digit>+
2. <number> ::= <digit> <number> | <digit>

Other complex inputs include arithmetic
1. fandango fuzz -f expr.fan -n 10
2. fandango fuzz -f expr-float.fan -n 10
3. fandango fuzz -f expr-float.fan -n 10 -c 'eval(str(<start>)) > 1000'

Divide by zero errors may occur




## Converting to the .fan specification

It is possible to utilize pre-existing ANTLR grammars to generate Fandango specifications. We can convert ANTLR .g4 files to .fan files using `fandango convert FILE.g4`, where `FILE.g4` is the  is the ANTLR file.

- Example: `fandango convert ./specs/name_db.g4`

## Interaction using Python API


Fandango also provides a limited Python API to perform basic tasks. Specifically, there are two methods, we can generate inputs (fuzz) and parse inputs with respect to a spec.

In [7]:
from fandango import Fandango

from constants import NAMEDB_SPEC

with open(NAMEDB_SPEC, "r") as spec:
    fan = Fandango(spec)

result = fan.fuzz(desired_solutions=1)[0]
result

Exception ignored in: <function BaseEventLoop.__del__ at 0x0000022722858A40>
Traceback (most recent call last):
  File "C:\Users\George\AppData\Local\Programs\Python\Python313\Lib\asyncio\base_events.py", line 763, in __del__
    if not self.is_closed():
  File "C:\Users\George\AppData\Local\Programs\Python\Python313\Lib\asyncio\base_events.py", line 760, in is_closed
    return self._closed
AttributeError: '_WindowsSelectorEventLoop' object has no attribute '_closed'


DerivationTree(<start>,
  [
    DerivationTree(<person_name>,
      [
        DerivationTree(<first_name>, [DerivationTree(<name>,
          [
            DerivationTree(<ascii_uppercase_letter>, [DerivationTree(<_ascii_uppercase_letter>, [DerivationTree('X')])]),
            DerivationTree(<ascii_lowercase_letter>, [DerivationTree(<_ascii_lowercase_letter>, [DerivationTree('m')])]),
            DerivationTree(<ascii_lowercase_letter>, [DerivationTree(<_ascii_lowercase_letter>, [DerivationTree('r')])]),
            DerivationTree(<ascii_lowercase_letter>, [DerivationTree(<_ascii_lowercase_letter>, [DerivationTree('q')])]),
            DerivationTree(<ascii_lowercase_letter>, [DerivationTree(<_ascii_lowercase_letter>, [DerivationTree('k')])]),
          ]
        )]),
        DerivationTree(' '),
        DerivationTree(<last_name>, [DerivationTree(<name>,
          [
            DerivationTree(<ascii_uppercase_letter>, [DerivationTree(<_ascii_uppercase_letter>, [DerivationTree('Z')])]),

In fact, the output is a parse tree with respect to the grammar we specified. Below, we can traverse the parsed result. We can access the raw input by invoking the string method `str(result)`.

In [8]:
from fandango import DerivationTree

def traverse_tree(node: DerivationTree, depth=0):
    indent = "  " * depth  # indentation for readability
    print(f"{indent} {node.symbol.format_as_spec()}")

    for child in node.children:
        traverse_tree(child, depth + 1)

for tree in fan.parse(result):
    traverse_tree(tree)

 <start>
   <person_name>
     <first_name>
       <name>
         <ascii_uppercase_letter>
           <_ascii_uppercase_letter>
             'X'
         <ascii_lowercase_letter>
           <_ascii_lowercase_letter>
             'm'
         <ascii_lowercase_letter>
           <_ascii_lowercase_letter>
             'r'
         <ascii_lowercase_letter>
           <_ascii_lowercase_letter>
             'q'
         <ascii_lowercase_letter>
           <_ascii_lowercase_letter>
             'k'
     ' '
     <last_name>
       <name>
         <ascii_uppercase_letter>
           <_ascii_uppercase_letter>
             'Z'
         <ascii_lowercase_letter>
           <_ascii_lowercase_letter>
             'r'
         <ascii_lowercase_letter>
           <_ascii_lowercase_letter>
             's'
         <ascii_lowercase_letter>
           <_ascii_lowercase_letter>
             'n'
         <ascii_lowercase_letter>
           <_ascii_lowercase_letter>
             'n'
         <ascii_lowerc

Finally, we can parse inputs to test whether they meet a specification

In [9]:
sample = "Lp Myei,55204"
p_sample = fan.parse(sample)  # returns a generator of DerivationTrees
for tree in p_sample:
    traverse_tree(tree)

 <start>
   <person_name>
     <first_name>
       <name>
         <ascii_uppercase_letter>
           <_ascii_uppercase_letter>
             'L'
         <ascii_lowercase_letter>
           <_ascii_lowercase_letter>
             'p'
     ' '
     <last_name>
       <name>
         <ascii_uppercase_letter>
           <_ascii_uppercase_letter>
             'M'
         <ascii_lowercase_letter>
           <_ascii_lowercase_letter>
             'y'
         <ascii_lowercase_letter>
           <_ascii_lowercase_letter>
             'e'
         <ascii_lowercase_letter>
           <_ascii_lowercase_letter>
             'i'
   ','
   <age>
     <digit>
       <_digit>
         '5'
     <digit>
       <_digit>
         '5'
     <digit>
       <_digit>
         '2'
     <digit>
       <_digit>
         '0'
     <digit>
       <_digit>
         '4'
